<a href="https://colab.research.google.com/github/surajsrivathsa/NL2SQL/blob/main/deepspeed_transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preamble

* Install Libraries and initial setup

In [1]:
!pip install datasets transformers rouge-score nltk

     |████████████████████████████████| 225kB 5.8MB/s 
     |████████████████████████████████| 2.1MB 26.9MB/s 
     |████████████████████████████████| 245kB 50.4MB/s 
     |████████████████████████████████| 112kB 49.5MB/s 
     |████████████████████████████████| 901kB 52.4MB/s 
     |████████████████████████████████| 3.3MB 49.8MB/s 


In [2]:
!pip install nvidia-ml-py3 

In [3]:
!pip install deepspeed
!pip install mpi4py

     |████████████████████████████████| 389kB 4.3MB/s 
     |████████████████████████████████| 225kB 36.6MB/s 
     |████████████████████████████████| 112kB 36.9MB/s 
  Created wheel for deepspeed: filename=deepspeed-0.3.15-cp37-none-any.whl size=386045 sha256=8f71a7ee101c10426835babfb2969169d26c679667db83da7db45ae91b6d0de9
  Stored in directory: /root/.cache/pip/wheels/e8/5e/3a/6d463184b2131b05d8f5cd20182cd02deb002a5924f7249763
Successfully built deepspeed
     |████████████████████████████████| 1.4MB 5.5MB/s 
  Created wheel for mpi4py: filename=mpi4py-3.0.3-cp37-cp37m-linux_x86_64.whl size=2065197 sha256=dbf002918d8d1f85bc28060c3b4982bc7bedf2be0549a6195493a52d7e29229a
  Stored in directory: /root/.cache/pip/wheels/18/e0/86/2b713dd512199096012ceca61429e12b960888de59818871d6
Successfully built mpi4py


# Revert to gpu version 10.1 to provide compatibility between deepspeeds compiled binaries(v10.1) vs local cuda(v11)

In [4]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:09_PDT_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.TC445_37.28845127_0


In [5]:
!ls /usr/local/

bin	   cuda-10.1  games		  lib	       man	  share
cuda	   cuda-11.0  _gcs_config_ops.so  LICENSE.txt  sbin	  src
cuda-10.0  etc	      include		  licensing    setup.cfg  xgboost


In [7]:
!stat /usr/local/cuda

  File: /usr/local/cuda -> cuda-11.0
  Size: 9         	Blocks: 0          IO Block: 4096   symbolic link
Device: 32h/50d	Inode: 1056905     Links: 1
Access: (0777/lrwxrwxrwx)  Uid: (    0/    root)   Gid: (    0/    root)
Access: 2021-04-21 13:32:00.000000000 +0000
Modify: 2021-04-21 13:32:00.000000000 +0000
Change: 2021-04-27 14:00:40.609379163 +0000
 Birth: -


In [8]:
%cat /usr/local/cuda/version.txt

CUDA Version 11.0.228


In [9]:
!rm -rf /usr/local/cuda

In [10]:
!ln -s /usr/local/cuda-10.1 /usr/local/cuda

In [11]:
!stat cuda

stat: cannot stat 'cuda': No such file or directory


# Setup Datasets and modelcheckpoints

In [12]:
model_checkpoint = "t5-small"

In [13]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [14]:
from datasets import load_dataset, load_metric

raw_datasets = load_dataset("xsum")
metric = load_metric("rouge")

Using custom data configuration default


Dataset xsum downloaded and prepared to /root/.cache/huggingface/datasets/xsum/default/1.2.0/4957825a982999fbf80bca0b342793b01b2611e021ef589fb7c6250b3577b499. Subsequent calls will reuse this data.


In [15]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 204045
    })
    validation: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 11332
    })
    test: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 11334
    })
})

In [16]:
raw_datasets["train"][1]

{'document': 'Army explosives experts were called out to deal with a suspect package at the offices on the Newtownards Road on Friday night.\nRoads were sealed off and traffic diverted as a controlled explosion was carried out.\nThe premises, used by East Belfast MP Naomi Long, have been targeted a number of times.\nMost recently, petrol bomb attacks were carried out on the offices on consecutive nights in April and May.\nThe attacks began following a Belfast City Council vote in December 2012 restricting the flying of the union flag at the City Hall.\nCondemning the latest hoax, Alliance MLA Chris Lyttle said: "It is a serious incident for the local area, it causes serious disruption, it puts people\'s lives at risk, it can prevent emergency services reaching the area.\n"Ultimately we need people with information to share that with the police in order for them to do their job and bring these people to justice."',
 'id': '28381580',
 'summary': 'A suspicious package left outside an All

In [17]:

import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [18]:
show_random_elements(raw_datasets["train"])

In [19]:
metric

Metric(name: "rouge", features: {'predictions': Value(dtype='string', id='sequence'), 'references': Value(dtype='string', id='sequence')}, usage: """
Calculates average rouge scores for a list of hypotheses and references
Args:
    predictions: list of predictions to score. Each predictions
        should be a string with tokens separated by spaces.
    references: list of reference for each prediction. Each
        reference should be a string with tokens separated by spaces.
    rouge_types: A list of rouge types to calculate.
        Valid names:
        `"rouge{n}"` (e.g. `"rouge1"`, `"rouge2"`) where: {n} is the n-gram based scoring,
        `"rougeL"`: Longest common subsequence based scoring.
        `"rougeLSum"`: rougeLsum splits text using `"
"`.
        See details in https://github.com/huggingface/datasets/issues/617
    use_stemmer: Bool indicating whether Porter stemmer should be used to strip word suffixes.
    use_agregator: Return aggregates if this is set to True
Retu

In [20]:
fake_preds = ["hello there", "general obi wan kenosha"]
fake_labels = ["hello there", "general obi kenobi"]
metric.compute(predictions=fake_preds, references=fake_labels)

{'rouge1': AggregateScore(low=Score(precision=0.5, recall=0.6666666666666666, fmeasure=0.5714285714285715), mid=Score(precision=0.75, recall=0.8333333333333333, fmeasure=0.7857142857142858), high=Score(precision=1.0, recall=1.0, fmeasure=1.0)),
 'rouge2': AggregateScore(low=Score(precision=0.3333333333333333, recall=0.5, fmeasure=0.4), mid=Score(precision=0.6666666666666666, recall=0.75, fmeasure=0.7), high=Score(precision=1.0, recall=1.0, fmeasure=1.0)),
 'rougeL': AggregateScore(low=Score(precision=0.5, recall=0.6666666666666666, fmeasure=0.5714285714285715), mid=Score(precision=0.75, recall=0.8333333333333333, fmeasure=0.7857142857142858), high=Score(precision=1.0, recall=1.0, fmeasure=1.0)),
 'rougeLsum': AggregateScore(low=Score(precision=0.5, recall=0.6666666666666666, fmeasure=0.5714285714285715), mid=Score(precision=0.75, recall=0.8333333333333333, fmeasure=0.7857142857142858), high=Score(precision=1.0, recall=1.0, fmeasure=1.0))}

In [21]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [22]:
if model_checkpoint in ["t5-small", "t5-base", "t5-larg", "t5-3b", "t5-11b"]:
    prefix = "summarize: "
else:
    prefix = ""

In [23]:
max_input_length = 1024
max_target_length = 128

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["document"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summary"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [24]:
newtraindataset = raw_datasets["train"].select([i for i in range(10000)])
newvalidationdataset = raw_datasets["validation"].select([i for i in range(500)])
newtestdataset = raw_datasets["test"].select([i for i in range(500)])

In [25]:
new_raw_dataset = datasets.dataset_dict.DatasetDict({"train": newtraindataset, "validation": newvalidationdataset, "test": newtestdataset})

In [26]:
preprocess_function(new_raw_dataset['train'][:2])

{'input_ids': [[21603, 10, 17716, 2279, 43, 5229, 128, 1410, 18, 390, 1508, 28, 5146, 12, 10256, 5, 96, 196, 31, 162, 373, 1800, 3, 18, 11, 48, 19, 28, 82, 22209, 3, 547, 30, 230, 117, 48, 19, 59, 1719, 42, 549, 8503, 3, 18, 27, 31, 26, 1066, 1492, 24, 540, 30, 2627, 1508, 16, 10256, 976, 243, 28571, 5, 37, 549, 8503, 795, 17586, 51, 12, 8, 3069, 11, 3996, 13606, 51, 639, 45, 8, 6266, 5, 18263, 10256, 11, 2390, 11, 7262, 10371, 7, 3971, 18, 17114, 28571, 1632, 549, 8503, 13404, 30, 2818, 1401, 1797, 6, 7229, 53, 20, 12151, 1955, 8356, 49, 53, 826, 3, 19585, 643, 9768, 5, 216, 19, 230, 3122, 3, 9, 2103, 1059, 12, 1175, 112, 1075, 38, 24260, 350, 16103, 10282, 7, 5752, 4297, 227, 271, 3, 11060, 30, 12, 8, 549, 8503, 1476, 16, 1600, 5, 28571, 47, 859, 8, 1374, 5638, 859, 10282, 7, 6, 411, 7, 2026, 63, 7, 6, 14586, 7677, 11, 26911, 2419, 7, 4298, 113, 130, 10960, 52, 26786, 16, 3, 9, 813, 11674, 11044, 28, 8, 549, 8503, 24, 3492, 16, 3, 9, 3996, 3328, 51, 1154, 16, 1660, 48, 215, 5, 86, 8,

In [27]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 204045
    })
    validation: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 11332
    })
    test: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 11334
    })
})

In [28]:
new_raw_dataset

DatasetDict({
    train: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 10000
    })
    validation: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 500
    })
    test: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 500
    })
})

In [29]:
tokenized_datasets = new_raw_dataset.map(preprocess_function, batched=True)

# TToo large of dataset, reduce size for checking
* Training will have 1000 samples and testing 50 samples

In [30]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'document', 'id', 'input_ids', 'labels', 'summary'],
        num_rows: 10000
    })
    validation: Dataset({
        features: ['attention_mask', 'document', 'id', 'input_ids', 'labels', 'summary'],
        num_rows: 500
    })
    test: Dataset({
        features: ['attention_mask', 'document', 'id', 'input_ids', 'labels', 'summary'],
        num_rows: 500
    })
})

In [ ]:
validation_dataset = tokenized_datasets["validation"]

In [ ]:
mylst = [i for i in range(1000)]

In [ ]:
mylst = [i for i in range(1000)]
mydataset = validation_dataset.select(mylst)

In [ ]:
newtraindataset = tokenized_datasets["train"].select([i for i in range(1000)])
newvalidationdataset = tokenized_datasets["validation"].select([i for i in range(50)])
newtestdataset = tokenized_datasets["test"].select([i for i in range(50)])

In [ ]:
new_tokenized_datasets = {"train": newtraindataset, "validation": newvalidationdataset, "test": newtestdataset}

In [ ]:
new_tokenized_datasets

{'test': Dataset({
     features: ['attention_mask', 'document', 'id', 'input_ids', 'labels', 'summary'],
     num_rows: 30
 }), 'train': Dataset({
     features: ['attention_mask', 'document', 'id', 'input_ids', 'labels', 'summary'],
     num_rows: 200
 }), 'validation': Dataset({
     features: ['attention_mask', 'document', 'id', 'input_ids', 'labels', 'summary'],
     num_rows: 30
 })}

In [ ]:
for i in range(4):
  print("Number of tokens in summary label, input big para and attention mask are : {}, {}, {}".format(len(tokenized_datasets["train"][i]["labels"]), len(tokenized_datasets["train"][i]["input_ids"]), len(tokenized_datasets["train"][i]["attention_mask"])))



Number of tokens in summary label, input big para and attention mask are : 46, 816, 816
Number of tokens in summary label, input big para and attention mask are : 22, 197, 197
Number of tokens in summary label, input big para and attention mask are : 24, 680, 680
Number of tokens in summary label, input big para and attention mask are : 19, 167, 167


# Model Training

In [31]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [34]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [35]:
import nltk
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

# Without Deepspeed

12 seconds per epoch for batch size of 16

In [ ]:
batch_size = 16
args = Seq2SeqTrainingArguments(
    "test-summarization",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=True,
)

In [ ]:

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=new_tokenized_datasets["train"],
    eval_dataset=new_tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)


Works well with batch sizes less than 16 , tried for 2, 8, 16. As batch size increases then it might not work in single GPU

* Speed of 1 second per epoch for 16 batch run

In [ ]:
import time
start = time.time()
trainer.train()
stop = time.time()
print(f"Training time: {stop - start}s")

Epoch,Training Loss,Validation Loss


LookupError: ignored

Checking GPU Memory

In [ ]:
import nvidia_smi

nvidia_smi.nvmlInit()

handle = nvidia_smi.nvmlDeviceGetHandleByIndex(0)
# card id 0 hardcoded here, there is also a call to get all available card ids, so we could iterate

info = nvidia_smi.nvmlDeviceGetMemoryInfo(handle)

print("Total memory:", info.total)
print("Free memory:", info.free)
print("Used memory:", info.used)

Total memory: 17071734784
Free memory: 98304000
Used memory: 16973430784


# With Deepspeed

In [32]:
deep_speed_config = {
    "fp16": {
        "enabled": True,
        "loss_scale": 0,
        "loss_scale_window": 1000,
        "hysteresis": 2,
        "min_loss_scale": 1
    },

   "zero_optimization": {
       "stage": 2,
       "allgather_partitions": True,
       "allgather_bucket_size": 5e8,
       "overlap_comm": True,
       "reduce_scatter": True,
       "reduce_bucket_size": 5e8,
       "contiguous_gradients": True,
       "cpu_offload": True
   },

   "optimizer": {
     "type": "AdamW",
     "params": {
       "lr": 2e-5,
       "betas": [ 0.8, 0.999 ],
       "eps": 1e-8,
       "weight_decay": 3e-7
     }
   },

   "scheduler": {
     "type": "WarmupLR",
     "params": {
       "warmup_min_lr": 0,
       "warmup_max_lr": 3e-5,
       "warmup_num_steps": 500
     }
   }
}

In [36]:
batch_size = 4
args = Seq2SeqTrainingArguments(
    "test-summarization",
    evaluation_strategy = "epoch",
    #learning_rate=2e-5,
    #weight_decay=0.001,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    deepspeed = deep_speed_config 
)

In [37]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)


In [38]:
import time
start = time.time()
trainer.train()
stop = time.time()
print(f"Training time: {stop - start}s")

[2021-04-28 20:52:14,012] [INFO] [logging.py:60:log_dist] [Rank 0] DeepSpeed info: version=0.3.15, git-hash=unknown, git-branch=unknown
[2021-04-28 20:52:14,017] [WARNING] [config.py:78:_sanity_check] DeepSpeedConfig: cpu_offload is deprecated. Please use offload_optimizer.
[2021-04-28 20:52:14,038] [INFO] [engine.py:80:_initialize_parameter_parallel_groups] data_parallel_size: 1, parameter_parallel_size: 1
Using /root/.cache/torch_extensions as PyTorch extensions root...
Creating extension directory /root/.cache/torch_extensions/cpu_adam...
Detected CUDA files, patching ldflags
Emitting ninja build file /root/.cache/torch_extensions/cpu_adam/build.ninja...
Building extension module cpu_adam...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
Loading extension module cpu_adam...
Time to load cpu_adam op: 39.62820243835449 seconds
[2021-04-28 20:52:55,786] [INFO] [engine.py:615:_configure_optimizer] Using DeepSpeed Optimiz

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len,Runtime,Samples Per Second
1,2.815400,2.556641,27.273200,6.632000,21.206900,21.184500,18.772000,93.027500,5.375000


[2021-04-28 20:53:17,324] [INFO] [stage2.py:1392:step] [deepspeed] fp16 dynamic loss scale overflow! Rank 0 Skipping step. Attempted loss scale: 4294967296, reducing to 2147483648.0
[2021-04-28 20:53:17,791] [INFO] [stage2.py:1392:step] [deepspeed] fp16 dynamic loss scale overflow! Rank 0 Skipping step. Attempted loss scale: 2147483648.0, reducing to 1073741824.0
[2021-04-28 20:53:18,179] [INFO] [stage2.py:1392:step] [deepspeed] fp16 dynamic loss scale overflow! Rank 0 Skipping step. Attempted loss scale: 1073741824.0, reducing to 536870912.0
[2021-04-28 20:53:18,439] [INFO] [stage2.py:1392:step] [deepspeed] fp16 dynamic loss scale overflow! Rank 0 Skipping step. Attempted loss scale: 536870912.0, reducing to 268435456.0
[2021-04-28 20:53:18,837] [INFO] [stage2.py:1392:step] [deepspeed] fp16 dynamic loss scale overflow! Rank 0 Skipping step. Attempted loss scale: 268435456.0, reducing to 134217728.0
[2021-04-28 20:53:19,118] [INFO] [stage2.py:1392:step] [deepspeed] fp16 dynamic loss sc

In [ ]:
import time
start = time.time()
trainer.train()
stop = time.time()
print(f"Training time: {stop - start}s")

[2021-04-28 20:21:47,471] [INFO] [logging.py:60:log_dist] [Rank 0] DeepSpeed info: version=0.3.15, git-hash=unknown, git-branch=unknown
[2021-04-28 20:21:47,477] [WARNING] [config.py:78:_sanity_check] DeepSpeedConfig: cpu_offload is deprecated. Please use offload_optimizer.
[2021-04-28 20:21:47,539] [INFO] [engine.py:80:_initialize_parameter_parallel_groups] data_parallel_size: 1, parameter_parallel_size: 1


Exception: ignored